In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.settings.settings import Settings

In [5]:
import sys

sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from chromadb.api.models.Collection import Collection
from typing import Optional
from src.utils.utils import clean_sentence

sql_collection = Settings.Chroma.get_sql_examples_collection()

def query_by_vector_embedding(
    collection: Collection,
    vector_embedding: list[float],
    n: Optional[int] = 5,
    score_threshold: Optional[float] = 0.2,
):
    
    results = collection.query(
        query_embeddings=vector_embedding,
        n_results=n,
        include=["distances", "metadatas"],
    )

    data = set()

    for distances, metadatas in zip(results["distances"], results["metadatas"]):
        for distance, metadata in zip(distances, metadatas):
            if distance <= score_threshold:
                metadata_tuple = tuple(metadata.items())
                data.add(metadata_tuple)

    return tuple(data)

In [20]:
from src.components.models.models import Openai_Model
text = "The human is requesting list of measurement systems"

model = Openai_Model()
model.init_model()

vector = model.get_embeddings(text)
# res = query_by_vector_embedding(sql_collection, vector)

In [22]:
from src.db.handlers.handlers import query_by_texts

res = query_by_texts(sql_collection, [text])

In [23]:
res

((('answer',
   "SELECT med.Nombre AS 'Measurement System Name', flu_tipo_fluido.Nombre AS 'Fluid Type'  FROM dbo_v2.med_sistema_medicion AS med  INNER JOIN dbo_v2.flu_tipo_fluido ON med.IdTipoFluido_fk = flu_tipo_fluido.Id  WHERE med.SubTipoFluido = 'DIF' AND flu_tipo_fluido.Nombre IS NOT NULL;"),
  ('question',
   'The human is requesting a list of measurement systems that are capable of reading differential gas fluid type.')),
 (('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion;'),
  ('question', 'The human is asking for a list of measurement systems.')),
 (('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion;'),
  ('question', 'The human is asking for a list of measurement systems')),
 (('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion WHERE Tag = <Specific Tag>;'),
  ('question

In [25]:
from src.db.handlers.handlers import query_by_vector_embedding
res = query_by_vector_embedding(sql_collection, vector)

In [26]:
res

((('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion;'),
  ('question', 'The human is asking for a list of measurement systems'),
  ('distance', 0.8346868104616998)),
 (('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion;'),
  ('question', 'The human is asking for a list of measurement systems.'),
  ('distance', 0.8346868104616998)),
 (('answer',
   'SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion WHERE Tag = <Specific Tag>;'),
  ('question',
   'The human is requesting information from a specific measurement system.'),
  ('distance', 0.8193098250407786)))